In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2

from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import os
import seaborn as sns


In [2]:
from keras.models import Sequential  
from keras.layers import Dense,Flatten,Dropout  
from keras.layers.convolutional import Conv2D,MaxPooling2D  
from keras.utils.np_utils import to_categorical  
import numpy as np  

In [3]:
seed = 7  
np.random.seed(seed)

In [4]:
print(os.listdir("images/"))

SIZE = 224  #Resize images

#Capture training data and labels into respective lists
train_images = []
train_labels = [] 

for directory_path in glob.glob("images/train/*"):
    label = directory_path.split("\\")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)
        
        #Convert lists to arrays        
train_images = np.array(train_images)
train_labels = np.array(train_labels)


# Capture test/validation data and labels into respective lists

test_images = []
test_labels = [] 
for directory_path in glob.glob("images/test/*"):
    fruit_label = directory_path.split("\\")[-1]
    print(fruit_label)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        test_images.append(img)
        test_labels.append(fruit_label)
        
        #Convert lists to arrays                
test_images = np.array(test_images)
test_labels = np.array(test_labels)

#Encode labels from text to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)
#it make the data calass name to numerical formate like 1,2,3,4 for calculation purpose

#Split data into test and train datasets (already split but assigning to meaningful convention)
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

# Normalize pixel values to between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

#One hot encode y values for neural network. make tahe numamic class in a matrix
from keras.utils.np_utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

['Test', 'Train']
Bacterial leaf blight
images/train\Bacterial leaf blight\DSC_0366.jpg
images/train\Bacterial leaf blight\DSC_0367.JPG
images/train\Bacterial leaf blight\DSC_0370.jpg
images/train\Bacterial leaf blight\DSC_0372.JPG
images/train\Bacterial leaf blight\DSC_0373.JPG
images/train\Bacterial leaf blight\DSC_0375.JPG
images/train\Bacterial leaf blight\DSC_0376.JPG
images/train\Bacterial leaf blight\DSC_0377.JPG
images/train\Bacterial leaf blight\DSC_0378.JPG
images/train\Bacterial leaf blight\DSC_0380.JPG
images/train\Bacterial leaf blight\DSC_0381.JPG
images/train\Bacterial leaf blight\DSC_0382.JPG
images/train\Bacterial leaf blight\DSC_0384.JPG
images/train\Bacterial leaf blight\DSC_0386.JPG
images/train\Bacterial leaf blight\DSC_0388.JPG
images/train\Bacterial leaf blight\DSC_0389.JPG
images/train\Bacterial leaf blight\DSC_0390.JPG
images/train\Bacterial leaf blight\DSC_0393.JPG
images/train\Bacterial leaf blight\DSC_0396.JPG
images/train\Bacterial leaf blight\DSC_0397.JPG


In [5]:
###########################################################################################################
#Load model wothout classifier/fully connected layers...thats means dont use the dense leyar
model = Sequential()  
model.add(Conv2D(96,(7,7),strides=(2,2),input_shape=(224,224,3),padding='valid',activation='relu',kernel_initializer='uniform'))  
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))  
model.add(Conv2D(256,(5,5),strides=(2,2),padding='same',activation='relu',kernel_initializer='uniform'))  
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))  
model.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))  
model.add(Flatten())  
model.add(Dense(4096,activation='relu'))  
model.add(Dropout(0.5))  
model.add(Dense(4096,activation='relu'))  
model.add(Dropout(0.5))  
model.add(Dense(1000,activation='softmax'))  
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])  
model.summary()
############################################################################################################

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 109, 109, 96)      14208     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 54, 54, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 384)       885120    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 384)       1327488   
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 256)       8

In [6]:
#Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights .. 
#no data will be trainable (we are not gone train this with multiple epocs  )

for layer in model.layers:
	layer.trainable = False
    
model.summary()  #Trainable parameters will be 0

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 109, 109, 96)      14208     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 54, 54, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 384)       885120    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 384)       1327488   
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 256)       8

In [7]:
from keras.models import Sequential  
from keras.layers import Dense,Flatten,Dropout  
from keras.layers.convolutional import Conv2D,MaxPooling2D  
from keras.utils.np_utils import to_categorical  
import numpy as np 

In [8]:
###########################################################################################################
#Load model wothout classifier/fully connected layers...thats means dont use the dense leyar
model1 = Sequential()  
model1.add(Conv2D(96,(11,11),strides=(4,4),input_shape=(224,224,3),padding='valid',activation='relu',kernel_initializer='uniform'))  
model1.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))  
model1.add(Conv2D(256,(5,5),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model1.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))  
model1.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model1.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model1.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model1.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))  
model1.add(Flatten())  
model1.add(Dense(4096,activation='relu'))  
model1.add(Dropout(0.5))  
model1.add(Dense(4096,activation='relu'))  
model1.add(Dropout(0.5))  
model1.add(Dense(1000,activation='softmax'))  
model1.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])  
model1.summary()
############################################################################################################

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 54, 54, 96)        34944     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 26, 26, 96)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 26, 26, 256)       614656    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 12, 12, 384)       885120    
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 12, 12, 384)       1327488   
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 12, 12, 256)      

In [9]:
#Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights .. 
#no data will be trainable (we are not gone train this with multiple epocs  )

for layer in model1.layers:
	layer.trainable = False
    
model1.summary()  #Trainable parameters will be 0

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 54, 54, 96)        34944     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 26, 26, 96)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 26, 26, 256)       614656    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 12, 12, 384)       885120    
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 12, 12, 384)       1327488   
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 12, 12, 256)      

In [10]:
#model
feature_extractor=model.predict(x_train)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

In [11]:
#model1
feature_extractor1=model1.predict(x_train)
features1 = feature_extractor.reshape(feature_extractor.shape[0], -1)

In [13]:
X_for_RF = np.concatenate((features, features1 ))

In [15]:
print(X_for_RF)

[[0.00073163 0.00074132 0.00093784 ... 0.00100307 0.00088781 0.00113183]
 [0.00072231 0.00065734 0.00096468 ... 0.00099668 0.00087783 0.00114559]
 [0.00071931 0.00078228 0.00095321 ... 0.00094807 0.00084162 0.00116955]
 ...
 [0.00071343 0.00077572 0.00111229 ... 0.00088565 0.00083252 0.00118155]
 [0.00082854 0.00088897 0.00093306 ... 0.00095004 0.00085873 0.00103908]
 [0.00085196 0.0008819  0.00107464 ... 0.00091182 0.00093631 0.00103955]]


In [16]:
#############################################################################################################################################
#RANDOM FOREST (it is classifiare problem not a regrassion problem nost use random forest regrassion) # n_estimators 50 mens it will generate 50 tree
# you can use another classifire module there like decition tree svm etc
from sklearn.ensemble import RandomForestClassifier 
RF_model = RandomForestClassifier(n_estimators = 50, random_state = 42)
###############################################################################################################################################

In [18]:
y_train= np.concatenate((y_train, y_train ))

In [20]:
# Train the model on training data
#For sklearn no one hot encoding x_for_rf random forest y_train (are level value)
RF_model.fit(X_for_RF, y_train) 


RandomForestClassifier(n_estimators=50, random_state=42)

In [21]:
X_test_feature =model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

In [22]:
X_test_feature1 =model1.predict(x_test)
X_test_features1 = X_test_feature.reshape(X_test_feature.shape[0], -1)

In [23]:
X_test_features= np.concatenate((X_test_features, X_test_features1 ))

In [24]:
#Now predict using the trained RF model. 
prediction_RF = RF_model.predict(X_test_features) # out put will look like 1.2.3

#Inverse le transform to get original label back. 
prediction_RF = le.inverse_transform(prediction_RF) #before we make the class value like 1,2,3 numeric its re build this

In [ ]:
test_labels= np.concatenate((test_labels, test_labels ))

In [25]:
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction_RF)) #provide test and prediction data to measure that

ValueError: Found input variables with inconsistent numbers of samples: [24, 48]